In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import imodelsx.process_results
import sys
import datasets
import numpy as np
from copy import deepcopy
import clin.eval
import clin.modules.prune
import clin.modules.evidence
import clin.modules.omission
import clin.modules.status
import clin.llm
import clin.parse
from collections import defaultdict
import openai
openai.api_key_path = '/home/chansingh/.OPENAI_KEY'

sys.path.append('../experiments/')
results_dir = '../results/'

r = imodelsx.process_results.get_results_df(results_dir, use_cached=False)

# get data for eval
dset = datasets.load_dataset('mitclinicalml/clinical-ie', 'medication_status')
df_val = pd.DataFrame.from_dict(dset['validation'])
df = pd.DataFrame.from_dict(dset['test'])
# df = pd.concat([val, test])
nums = np.arange(len(df)).tolist()
np.random.default_rng(seed=13).shuffle(nums)
dfe = df.iloc[nums]
n = len(dfe)

# single row to investigate
row = r[(r.n_shots == 5) * (r.checkpoint == 'text-davinci-003')].iloc[0]
sv = clin.modules.status.StatusVerifier()

# (
#     r.groupby(['checkpoint', 'n_shots'])[['f1', 'recall', 'precision']].mean()
#     .style.format(precision=3).background_gradient(cmap='Blues')
# )

In [ ]:
# compare values for a single row
row_df = r[(r.n_shots == 5) * (r.checkpoint == 'text-davinci-003')]
rc = row_df[[c for c in row_df.columns if '___' in c]]
# create multindex columns by splitting on '___'
rc = rc.rename(columns={c: tuple(c.split('___')) for c in rc.columns})

# convert tuple column names to multiindex
rc.columns = pd.MultiIndex.from_tuples(rc.columns)
rc = rc.T.reset_index()
rc = rc.rename(columns={
    'level_0': 'metric',
    'level_1': 'verification',
}).pivot_table(index='verification', columns='metric', values=0).round(3)
rc.style.format(precision=3).background_gradient(cmap='Blues')

In [ ]:
# print errs
# mets = clin.eval.calculate_metrics(row['dict_ov_pv_ev'], dfe, verbose=True)

### Test on medication status

In [ ]:
settings = [k for k in row.keys() if k.startswith('dict_') and not k.startswith('dict_evidence')]
med_status_dicts_list = [row[k] for k in settings]

In [ ]:
# add medication status eval
# med_status_dicts_list = [[clin.parse.parse_response_medication_list(r.iloc[i]['resps'][j]) for j in range(n)] for i in range(len(r))]
d = defaultdict(list)
accs_cond, f1s_macro_cond = clin.eval.eval_medication_status(med_status_dicts_list, dfe)
d['acc_cond'] = accs_cond
d['f1_macro_cond'] = f1s_macro_cond
d['index'] = settings
(
    pd.DataFrame.from_dict(d).set_index('index') #[['acc_cond', 'f1_macro_cond']]
    .style.format(precision=3).background_gradient(cmap='Blues')
)

In [ ]:
# viz errs
# accs_cond, f1s_macro_cond = clin.eval.eval_medication_status([med_status_dicts_list[-1]], dfe, verbose=True)